In [1]:
%pip install --upgrade pip
%pip install --disable-pip-version-check\
  torch==1.13.1\
  torchdata==0.5.1 --quiet
%pip install \
  transformers==4.27.2 \
  datasets==2.11.0 --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install \
    evaluate==0.4.0 \
    rouge_score==0.1.2 \
    loralib==0.1.1 \
    peft==0.3.0 --quiet

Note: you may need to restart the kernel to use updated packages.


In [3]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments,Trainer
import torch
import time
import evaluate 
import pandas as pd
import numpy as np

/Users/amiteshgangrade/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load Dataset and LLM

In [4]:
huggingface_dataset_name = "knkarthick/dialogsum"

dataset = load_dataset(huggingface_dataset_name)

Found cached dataset csv (/Users/amiteshgangrade/.cache/huggingface/datasets/knkarthick___csv/knkarthick--dialogsum-c8fac5d84cd35861/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
100%|████████████████████████████████████████████| 3/3 [00:00<00:00, 172.50it/s]


In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
})

In [6]:
model_name = 'google/flan-t5-base'

original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

 Lets check all the trainable model parameters

In [7]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f'trainble model parameters : {trainable_model_params}\nall model parameters : {all_model_params}\n'


print(print_number_of_trainable_model_parameters(original_model))

trainble model parameters : 247577856
all model parameters : 247577856



### Test Model with Zero Shot Inference 

In [8]:
index = 200

dialogue = dataset['test'][index]['dialogue']
summary = dataset['test'][index]['summary']

prompt = f"""
Summarize the following conversation.
{dialogue}

Summary: 
"""

inputs = tokenizer(prompt,return_tensors='pt')
output = tokenizer.decode(
    original_model.generate(
        inputs['input_ids'],
        max_new_tokens=200,
    )[0],
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'Input Prompt:\n{prompt}')
print(dash_line)
print(f'Baseline Human Summary : \n{summary}')
print(dash_line)
print(f'Model Generation - Zero Shot Inference: \n{output}')

---------------------------------------------------------------------------------------------------
Input Prompt:

Summarize the following conversation.
#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also need a more powerful hard disc, more memory and a faster modem. Do you have a CD-ROM drive?
#Person2#: No.
#Person1#: Then you might want to add a CD-ROM drive too, because most new software programs are coming out on Cds.
#Person2#: That sounds great. Thanks.

Summary: 

-------------------------------------------------------------------

## Full Fine-Tuning 

#### Preprocess the Dialogue-Summary Dataset

In [12]:
def tokenize_function(example):
    start_prompt = "Summarize the following conversation.\n\n"
    end_prompt = '\n\nSummary:'
    prompt = [start_prompt + dialogue + end_prompt for dialogue in example['dialogue']]
    example['input_ids'] = tokenizer(prompt,padding='max_length',truncation=True,return_tensors='pt').input_ids
    example['labels'] = tokenizer(example['summary'],padding='max_length',truncation=True,return_tensors='pt').input_ids
    return example

tokenized_datasets = dataset.map(tokenize_function,batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['id','topic','dialogue','summary'])


Subsampling the dataset

In [13]:
tokenized_datasets = tokenized_datasets.filter(lambda example,index:index%100==0,with_indices=True)

In [15]:
print(f"Shapes of the datasets")
print(f"Training : {tokenized_datasets['train'].shape}")
print(f"Validation : {tokenized_datasets['validation'].shape}")
print(f"Test : {tokenized_datasets['test'].shape}")

print(tokenized_datasets)

Shapes of the datasets
Training : (125, 2)
Validation : (5, 2)
Test : (15, 2)
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 125
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 15
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 5
    })
})


### Fine-Tune the Model with Preprocessed Dataset

Now utilize the built-in Higging-face Trainer class. Pass the preprocessed Dataset with reference to the original model.Other training parameters are found experimentally. 

In [17]:
output_dir = f'./dialogue-summary-training-{str(int(time.time()))}'

training_args = TrainingArguments(
    output_dir = output_dir,
    learning_rate=1e-5,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_steps=1,
    max_steps=1
    )

trainer = Trainer(
    model = original_model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation']
)

In [ ]:
trainer.train()

/Users/amiteshgangrade/anaconda3/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
